# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function - Start ----------\n")

In [ ]:
medallion_name = "Bronze"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

# Function

## Get existing technology list

In [ ]:
def fn_get_existing_technology_list(list_frequency):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        sql_code = f"""
SELECT DISTINCT       `technology` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_csv`        WHERE `is_extracted` = 1 AND `frequency` IN ({str(list_frequency).replace("[", "").replace("]", "")})
UNION SELECT DISTINCT `technology` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_excel`      WHERE `is_extracted` = 1 AND `frequency` IN ({str(list_frequency).replace("[", "").replace("]", "")})
UNION SELECT DISTINCT `technology` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_json`       WHERE `is_extracted` = 1 AND `frequency` IN ({str(list_frequency).replace("[", "").replace("]", "")})
UNION SELECT DISTINCT `technology` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`  WHERE `is_extracted` = 1 AND `frequency` IN ({str(list_frequency).replace("[", "").replace("]", "")})
UNION SELECT DISTINCT `technology` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server` WHERE `is_extracted` = 1 AND `frequency` IN ({str(list_frequency).replace("[", "").replace("]", "")})
UNION SELECT DISTINCT `technology` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_api`        WHERE `is_extracted` = 1 AND `frequency` IN ({str(list_frequency).replace("[", "").replace("]", "")});
        """
        rv              = fn_execute_spark_sql(sql_code)
        sdf             = rv[2]
        sdf_count       = rv[3]
        list_technology = [row.technology for row in sdf.collect()]

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        alert             = "Success"
        alert_description = f"Technology list: {list_technology}"

        return (alert, alert_description, list_technology, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

In [ ]:
# Get Existing Technology List
list_technology = fn_get_existing_technology_list(list_frequency)[2]
if is_debug: print(f"list_technology: {list_technology}")

## Get a list of the extracted tables

In [ ]:
def fn_get_extract_table_list(dict_parameter):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        if \
            dict_parameter["technology"] == "Excel" \
            and dict_parameter["action"] == "Extract":

            sql_query = f"""SELECT DISTINCT
    `technology`
    , `frequency`
    , IFNULL(`folder_name`   , \"\") AS `folder_name`
    , IFNULL(`file_name`     , \"\") AS `file_name`
    , IFNULL(`worksheet_name`, \"\") AS `worksheet_name`
    , `first_row`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_excel`
WHERE
    `frequency`        = \"{dict_parameter["frequency"]}\"
    AND `is_extracted` = 1
ORDER BY
    `technology`
    , `frequency`
    , IFNULL(`folder_name`   , \"\")
    , IFNULL(`file_name`     , \"\")
    , IFNULL(`worksheet_name`, \"\");"""

        elif \
            dict_parameter["technology"] == "Excel" \
            and dict_parameter["action"] == "Archive":

            sql_query = f"""SELECT DISTINCT
    `technology`
    , `frequency`
    , IFNULL(`folder_name`, \"\") AS `folder_name`
    , IFNULL(`file_name`  , \"\") AS `file_name`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_excel`
WHERE
    `frequency`        = \"{dict_parameter["frequency"]}\"
    AND `is_extracted` = 1
ORDER BY
    `frequency`
    , IFNULL(`folder_name`, \"\")
    , IFNULL(`file_name`  , \"\");"""

        elif \
            (dict_parameter["technology"] == "SQL Server" or dict_parameter["technology"] == "Lakehouse") \
            and dict_parameter["action"]  == "Extract":

            sql_query = (f"""SELECT DISTINCT
    A.`technology`
    , A.`frequency`
    , IFNULL(A.`lakehouse_name`               , \"\") AS `lakehouse_name`
    , IFNULL(A.`server_name`                  , \"\") AS `server_name`
    , IFNULL(A.`database_name`                , \"\") AS `database_name`
    , IFNULL(A.`schema_name`                  , \"\") AS `schema_name`
    , IFNULL(A.`table_name`                   , \"\") AS `table_name`
    , IFNULL(A.`keyvault_secret_name_user`    , \"\") AS `keyvault_secret_name_user`
    , IFNULL(A.`keyvault_url`                 , \"\") AS `keyvault_url`
    , IFNULL(A.`keyvault_secret_name_password`, \"\") AS `keyvault_secret_name_password`
FROM
    (
        SELECT `technology`, `frequency`, \"\" AS `lakehouse_name`, `server_name`, `database_name`, `schema_name`, `table_name`, `keyvault_secret_name_user`, `keyvault_url`, `keyvault_secret_name_password`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`
        UNION SELECT `technology`, `frequency`, `lakehouse_name`, \"\" AS `server_name`, \"\" AS `database_name`, \"\" AS `schema_name`, `table_name`, \"\" AS `keyvault_secret_name_user`, \"\" AS `keyvault_url`, \"\" AS `keyvault_secret_name_password`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
    ) AS A
WHERE
    A.`technology`      = \"{dict_parameter["technology"]}\"
    AND A.`frequency`   = \"{dict_parameter["frequency"]}\"
    AND A.`is_extracted`= 1
ORDER BY
    `technology`
    , `frequency`
    , IFNULL(`lakehouse_name`  , \"\")
    , IFNULL(`server_name`  , \"\")
    , IFNULL(`database_name`, \"\")
    , IFNULL(`schema_name`  , \"\")
    , IFNULL(`table_name`   , \"\");""")

#         elif \
#             dict_parameter["technology"] == "Lakehouse" \
#             and dict_parameter["action"] == "Extract":

#             sql_query = (f"""SELECT DISTINCT
#     `technology`
#     , `frequency`
#     , IFNULL(`lakehouse_name`, \"\") AS `lakehouse_name`
#     , IFNULL(`table_name`    , \"\") AS `table_name`
# FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
# WHERE
#     `frequency`       = \"{dict_parameter["frequency"]}\"
#     AND `is_extracted`= 1
# ORDER BY
#     `technology`
#     , `frequency`
#     , IFNULL(`lakehouse_name`, \"\")
#     , IFNULL(`table_name`    , \"\");""")

        elif \
            dict_parameter["technology"] == "CSV" \
            and (dict_parameter["action"] == "Extract" or dict_parameter["action"] == "Archive"):

            sql_query = (f"""SELECT DISTINCT
    `technology`
    , `frequency`
    , IFNULL(`folder_name`       , \"\") AS `folder_name`
    , IFNULL(`file_name`         , \"\") AS `file_name`
    , IFNULL(STRING(`has_header`), \"\") AS `has_header`
    , IFNULL(`delimiter`         , \"\") AS `delimiter`
    , IFNULL(`row_separator`     , \"\") AS `row_separator`
    , IFNULL(`quote`             , \"\") AS `quote`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_csv`
WHERE
    `frequency`        = \"{dict_parameter["frequency"]}\"
    AND `is_extracted` = 1
ORDER BY
    `technology`
    , `frequency`
    , IFNULL(`folder_name`, \"\")
    , IFNULL(`file_name` , \"\");""")

        elif \
            dict_parameter["technology"] == "JSON" \
            and (dict_parameter["action"] == "Extract" or dict_parameter["action"] == "Archive"):

            sql_query = (f"""SELECT DISTINCT
    `technology`
    , `frequency`
    , IFNULL(`folder_name`, \"\") AS `folder_name`
    , IFNULL(`file_name`  , \"\") AS `file_name`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_json`
WHERE
    `frequency`        = \"{dict_parameter["frequency"]}\"
    AND `is_extracted` = 1
ORDER BY
    `technology`
    , `frequency`
    , IFNULL(`folder_name`, \"\")
    , IFNULL(`file_name`  , \"\");""")

        sdf = fn_execute_spark_sql(sql_query)[2]
        sdf_count = sdf.count()

        alert             = "Success"
        alert_description = f"Table/file list (count): {sdf_count}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, sdf, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)        

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Get a list of the columns of the extracted tables

In [ ]:
def fn_get_extract_column(dict_parameter):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        if dict_parameter["technology"] == "SQL Server" or dict_parameter["technology"] == "Lakehouse":
            if dict_parameter["technology"] == "SQL Server": lakehouse_name = ""
            else:                                            lakehouse_name = dict_parameter["lakehouse_name"]
            
            if dict_parameter["technology"] == "SQL Server": server_alias = dict_parameter["server_alias"]
            else:                                            server_alias = ""

            if dict_parameter["technology"] == "SQL Server": database_name = dict_parameter["database_name"]
            else:                                            database_name = ""

            if dict_parameter["technology"] == "SQL Server": schema_name = dict_parameter["schema_name"]
            else:                                            schema_name = ""
            
            table_name = dict_parameter["table_name"]

            sql_query = f"""SELECT DISTINCT
    `sequence`
    , IFNULL(A.`prefix_select`, \"\")  AS `prefix_select`
    , IFNULL(A.`column_name`  , \"\")  AS `column_name`
    , TRIM(IFNULL(CONCAT(
        CASE WHEN A.`sequence` = 1 THEN '' ELSE ', ' END,
        CONCAT(IF(A.`prefix_select` IS NOT NULL, CONCAT(A.`prefix_select`, '.'), ''), '[', A.`column_name`, '] AS [', A.`column_name`, ']')
    )                         , \"\")) AS `alias`
    , IFNULL(A.`from`         , \"\")  AS `from`
    , IFNULL(A.`where`        , \"\")  AS `where`
    , A.`is_pk`                        AS `is_pk`
    , IFNULL(A.`data_type`    , \"\")  AS `data_type`
FROM
  (
          SELECT `technology`, `frequency`, `server_name`, `database_name`, `schema_name`, `table_name`, \"\" AS `lakehouse_name`, `sequence`, `prefix_select`, `column_name`, `from`, `where`, `is_pk`, `data_type`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server`
    UNION SELECT `technology`, `frequency`, \"\" AS `server_name`, \"\" AS `database_name`, \"\" AS `schema_name`, `table_name`, `lakehouse_name`, `sequence`, `prefix_select`, `column_name`, `from`, `where`, `is_pk`, `data_type`, `is_extracted` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`
  ) AS A
WHERE
    A.`technology`                       = \"{dict_parameter["technology"]}\"
    AND A.`frequency`                    = \"{dict_parameter["frequency"]}\"
    AND IFNULL(A.`lakehouse_name`, \"\") = \"{lakehouse_name}\"
    AND IFNULL(A.`server_name`   , \"\") = \"{server_alias}\"
    AND IFNULL(A.`database_name` , \"\") = \"{database_name}\"
    AND IFNULL(A.`schema_name`   , \"\") = \"{schema_name}\"
    AND IFNULL(A.`table_name`    , \"\") = \"{table_name}\"
    AND A.`is_extracted`                 = 1
ORDER BY CAST(`sequence` AS BIGINT);"""

        elif dict_parameter["technology"] == "Excel":
            sql_query = f"""SELECT
    `sequence`
    , IFNULL(`column_name`, \"\") AS `column_name`
    , `is_pk`
    , IFNULL(`data_type`  , \"\") AS `data_type`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_excel`
WHERE
    `technology`                       = \"{dict_parameter["technology"]}\"
    AND `frequency`                    = \"{dict_parameter["frequency"]}\"
    AND IFNULL(`folder_name`   , \"\") = \"{dict_parameter["folder_name"]}\"
    AND IFNULL(`file_name`     , \"\") = \"{dict_parameter["file_name"]}\"
    AND IFNULL(`worksheet_name`, \"\") = \"{dict_parameter["worksheet_name"]}\"
    AND `is_extracted`                 = 1
ORDER BY CAST(`sequence` AS BIGINT);"""

        elif dict_parameter["technology"] == "CSV":
            sql_query = f"""SELECT
    `sequence`
    , IFNULL(`column_name`  , \"\") AS `column_name`
    , `is_pk`
    , IFNULL(`data_type`    , \"\") AS `data_type`
    , `has_header`
    , IFNULL(`delimiter`    , \"\") AS `delimiter`
    , IFNULL(`row_separator`, \"\") AS `row_separator`
    , IFNULL(`quote`        , \"\") AS `quote`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_csv`
WHERE
    `technology`                       = \"{dict_parameter["technology"]}\"
    AND `frequency`                    = \"{dict_parameter["frequency"]}\"
    AND IFNULL(`folder_name`   , \"\") = \"{dict_parameter["folder_name"]}\"
    AND IFNULL(`file_name`     , \"\") = \"{dict_parameter["file_name"]}\"
    AND `is_extracted`                 = 1
ORDER BY CAST(`sequence` AS BIGINT);"""

        elif dict_parameter["technology"] == "JSON":
            sql_query = f"""SELECT
    `sequence`
    , IFNULL(`column_name`       , \"\") AS `column_name`
    , IFNULL(`parent_column_name`, \"\") AS `parent_column_name`
    , `is_pk`
    , IFNULL(`data_type`         , \"\") AS `data_type`
FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_json`
WHERE
    `technology`                       = \"{dict_parameter["technology"]}\"
    AND `frequency`                    = \"{dict_parameter["frequency"]}\"
    AND IFNULL(`folder_name`   , \"\") = \"{dict_parameter["folder_name"]}\"
    AND IFNULL(`file_name`     , \"\") = \"{dict_parameter["file_name"]}\"
    AND `is_extracted`                 = 1
ORDER BY CAST(`sequence` AS BIGINT);"""

        sdf       = fn_execute_spark_sql(sql_query)[2]
        sdf_count = sdf.count()

        alert             = "Success"
        alert_description = f"Column list (count): {sdf_count}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, sdf, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Dataframe

### Save in single .parquet file

In [ ]:
# Used for archiving
def fn_save_in_single_file(
    sdf
    , file_name
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        # Add column "process_timestamp"
        sdf       = sdf.withColumn("process_timestamp", sf.lit(global_parameter.process_timestamp))
        sdf_count = sdf.count()

        # Cast and collect all Timestamp columns to String
        columns = []
        for col in sdf.dtypes:
            if col[1] == "timestamp":
                sdf = sdf.withColumn(col[0], sf.date_format(col[0], "yyyy-MM-dd HH:mm:ss.SSSSSS"))
                columns.append(col[0])

        # Convert SDF to PDF
        pdf = sdf.toPandas()

        # Cast the collected Timestamp columns to Pandas DateTime
        for col in columns: pdf[col] = pd.to_datetime(pdf[col])
        
        # Save in single .parquet file
        pdf.to_parquet(file_name, index = False)

        alert             = "Success"
        alert_description = f"Saved in single file (name): {file_name}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par

        pdf = pdf.head(5)
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Operation

## Rename existing files in "lh_bronze"

In [ ]:
# Clean from filename:
#    Accents
#    All non-alphanumerical (keep only [A-Z][a-z][0-9]_)

if ("Excel" in list_technology or "CSV" in list_technology or "JSON" in list_technology):
    def fn_rename_files_in_lh_bronze(technology, frequency, action):
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}

        try:
            # Set file_extention
            if technology   == "Excel": file_extention = "xlsx"
            elif technology == "CSV":   file_extention = "csv"
            elif technology == "JSON":  file_extention = "json"

            # Get table list
            dict_parameter = {"technology": technology, "frequency": frequency, "action": action}
            sdf = fn_get_extract_table_list(dict_parameter)[2]
            sdf_count = sdf.count()

            # Loop archive table list
            for row in sdf \
                .sort(
                    sdf.technology.asc()
                    , sdf.frequency.asc()
                    , sdf.folder_name.asc()
                    , sdf.file_name.asc()
                ) \
                .collect():

                dict_parameter = {"technology": row.technology, "frequency": row.frequency, "folder_name": row.folder_name, "file_name": row.file_name}

                # Clean
                folder_name_clean = row.folder_name.replace("/", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789") # keep "/" while cleaning
                folder_name_clean = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(folder_name_clean)[2], "_")[2]
                folder_name_clean = folder_name_clean.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "/")
                file_name_clean   = fn_clean_not_alphanumeric_in_string(fn_clean_accent_in_string(row.file_name)[2], "_")[2]

                # Rename file
                dict_parameter["folder_name_clean"] = folder_name_clean
                dict_parameter["file_name_clean"]   = file_name_clean
                
                src = f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/{row.technology}/{row.folder_name}/{row.file_name}.{file_extention}"
                src = src.replace("abfss://", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
                src = src.replace("//", "/")
                src = src.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "abfss://")
                
                dstn = f"{global_parameter.abfs_path_lh_bronze}/Files/incoming/{row.technology}/{folder_name_clean}/{file_name_clean}.{file_extention}"
                dstn = dstn.replace("abfss://", "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789")
                dstn = dstn.replace("//", "/")
                dstn = dstn.replace("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", "abfss://")
                fn_move_file(src, dstn, dict_parameter)

            alert             = "Success"
            alert_description = f"Expected {technology} files to rename (count): {sdf_count}"            
            
            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, sdf)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                fn_print_debug_info(alert, fn_name, par)
                del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Create lookup SDF "server_alias - server_name"

In [ ]:
# Used to apply "SQL Server aliases"
if "SQL Server" in list_technology or "Lakehouse" in list_technology:
    def fn_create_lookup_server_alias_server_name():
        fn_name        = stk()[0][3]
        caller_fn_name = stk()[1].function.replace("<module>", "")
        if is_debug: par = {}
        
        try:
            sql_code = f"""SELECT DISTINCT
    EO.`server_name` AS `server_alias`
    , GP.`value`     AS `server_name`
FROM
    delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server` AS EO
    JOIN delta.`{global_parameter.abfs_path_lh_cfg}/Tables/global_parameter`     AS GP ON EO.`server_name` = GP.`name`
WHERE EO.`is_extracted` = 1;"""

            rv_lookup_server_alias_server_name = fn_execute_spark_sql(sql_code)
            sdf                                = rv_lookup_server_alias_server_name[2]
            sdf_count                          = rv_lookup_server_alias_server_name[3]

            dict_server_alias_server_name = {}
            for row in sdf.collect(): dict_server_alias_server_name[row.server_alias] = row.server_name

            alert             = "Success"
            alert_description = f"Lookup SDF \"server_alias - server_name\" (count): {sdf_count}"

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

            return (alert, alert_description, dict_server_alias_server_name, sdf_count)
        except Exception as ex:
            alert             = "Danger"
            alert_description = str(ex)

            return (alert, alert_description, None, None)
        finally:
            if is_debug:
                par["locals"] = locals()
                fn_print_debug_info(alert, fn_name, par)
                del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Extract Preparation

In [ ]:
def fn_extract_preparation(list_technology):   
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        # Clean up log
        fn_lh_log_log_cleanup(global_parameter.days_to_keep_log)

        # Insert SimpleNamespace.global_parameter in "lh_log.log"
        fn_local_log_insert_global_parameter()

        # Delete the tables in "lh_bronze"
        fn_delete_all_in_folder(f"{global_parameter.abfs_path_lh_bronze}/Tables")

        # Create lookup "server_alias - server_name"
        if "SQL Server" in list_technology: dict_server_alias_server_name = fn_create_lookup_server_alias_server_name()[2]

        alert             = "Success"
        alert_description = ""

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, dict_server_alias_server_name)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Extract

In [ ]:
def fn_bronze_extract():
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}
    
    try:
        # Prepare for extract
        #   Clean up lh_log.log
        #   Insert SimpleNamespace.global_parameter in "lh_log.log"
        #   Delete the tables in "lh_bronze"
        #   Create lookup "server_alias - server_name"

        # Extract preparation
        dict_server_alias_server_name = fn_extract_preparation(list_technology)[2]

        if is_debug:
            print(f"dict_server_alias_server_name (type): {type(dict_server_alias_server_name)}")
            print(f"dict_server_alias_server_name:        {dict_server_alias_server_name}")

        # Extract
        for row in sdf_technology_frequency.filter(sdf_technology_frequency.technology == "SQL Server").collect():
            fn_extract_lakehouse_or_sql_server(row.technology, row.frequency, dict_server_alias_server_name)
        
        for row in sdf_technology_frequency.filter(sdf_technology_frequency.technology == "Lakehouse").collect():
            fn_extract_lakehouse_or_sql_server(row.technology, row.frequency, dict_server_alias_server_name)
        
        for row in sdf_technology_frequency.filter(sdf_technology_frequency.technology == "Excel").collect():
            fn_extract_excel(row.technology, row.frequency)
        
        for row in sdf_technology_frequency.filter(sdf_technology_frequency.technology == "CSV").collect():
            fn_extract_csv(row.technology, row.frequency)
        
        for row in sdf_technology_frequency.filter(sdf_technology_frequency.technology == "JSON").collect():
            fn_extract_json(row.technology, row.frequency)
        
        #for row in sdf_technology_frequency.filter(sdf_technology_frequency.technology == "API").collect():
        #    fn_extract_api(row.technology, row.frequency)

        # Extract finalization
        # 1. Append to table "lh_log.extract"
        #fn_append_to_lh_log_extract() PL: Do this or generate report from lh_log.log?

        # 2. Update "datetime_from" and "datetime_to" in table "lh_cfg.extract_parameter" for the successfully extracted tables
        if "SQL Server" or "Lakehouse" in list_technology: fn_update_datetime_from_datetime_to_in_extract_parameter(pdf_log) # or "API"

        alert             = "Success"
        alert_description = ""

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
        
        # fn_print_debug_info_123("Success", fn_name, par) # test "except"
        
        # Append local log to lh_log.log
        fn_lh_log_log_insert()

        # Generate HTML table to be sent by email to global_parameter.email_on_error
        #fn_get_extract_report()

        return (alert, alert_description)

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_bronze_function - End ----------\n")